In [45]:
# First import the libraries that we need to use
import os
import pandas as pd
from datetime import datetime
import requests
import json
from utils.coinbase import fetch_product_trades
from dotenv import load_dotenv
load_dotenv()

cb_pro = os.getenv("COINBASE_PRO_FQDN")
cb_pub = os.getenv("COINBASE_PUB_FQDN")
cb_sandbox = os.getenv("COINBASE_SANDBOX_FQDN")

print(cb_pro)
print(cb_pub) # used for most calls
print(cb_sandbox)


https://api-pro.coinbase.com
https://api.exchange.coinbase.com
https://api-public.sandbox.exchange.coinbase.com


In [46]:
def get_currencies():
    """
    Returns DataFrame of currencies available on coinbase exchange.
    
    """
    coin_url = f"{cb_pub}/currencies"
    coin_response = requests.get(coin_url)
    if coin_response.status_code == 200:
        filename = "data/coinbase_currencies.csv"
        currencies_df = pd.DataFrame(json.loads(coin_response.text), columns=['id', 'name', 'min_size', 'status', 'max_precision']).fillna('')
        currencies_df.to_csv(filename, index=False)
        print(f"Created CSV file: {filename}")
    else:
        print(f"Did not receive 200 response")
 
    return currencies_df

currencies_df = get_currencies()


Created CSV file: data/coinbase_currencies.csv


In [47]:
currencies_df

,id,name,min_size,status,max_precision
0,RLC,iExec RLC,0.1,online,0.01
1,GODS,Gods Unchained,0.1,online,0.00000001
2,BAT,Basic Attention Token,1,online,0.00000001
3,BTC,Bitcoin,0.00000001,online,0.00000001
4,ICP,Internet Computer,0.01,online,0.001
...,...,...,...,...,...
173,MLN,Enzyme,0.01,online,0.001
174,UMA,UMA,0.01,online,0.001
175,BTRST,Braintrust,1,online,0.01
176,LCX,LCX Token,0.1,online,0.0001


In [48]:
# import requests
# url = "https://api.exchange.coinbase.com/products"
# headers = {"Accept": "application/json"}
# response = requests.get(url, headers=headers)
# # print(response.text)

In [49]:
## Get full list of Products and information
def get_products():
    """
    Returns a listing of data products that are available

    """
    product_url = f"{cb_pub}/products"
    headers = {"Accept": "application/json"}
    product_response = requests.get(product_url, headers=headers)
    product_columns=["id","base_currency","quote_currency","base_min_size","base_max_size","quote_increment","base_increment","display_name","min_market_funds","max_market_funds","margin_enabled","fx_stablecoin","max_slippage_percentage","post_only","limit_only","cancel_only","trading_disabled","status","status_message","auction_mode"]
    # if product_response.status.code == 200:
    filename = "data/coinbase_products.csv"
    products_df = pd.DataFrame(json.loads(product_response.text), columns=product_columns).filter(items=['id','base_currency','quote_currency','status'])
    ## Filter our delisted currency products
    products_df = products_df[~products_df['status'].isin(['delisted'])]
    products_df = products_df.drop(columns=["status"])
    ## Save a local copy
    products_df.to_csv(filename, index=False)
    print(f"Created CSV file: {filename}")
    # print(product_response.text)
    # else:
    # print(f"Did not receive 200 response")

    return products_df

products_online_df = get_products()


Created CSV file: data/coinbase_products.csv


In [50]:
# Show products that are online and available
products_online_df.head()

,id,base_currency,quote_currency
0,MIR-EUR,MIR,EUR
1,WBTC-USD,WBTC,USD
2,POWR-USDT,POWR,USDT
3,USDT-GBP,USDT,GBP
4,GYEN-USD,GYEN,USD


In [51]:
# Show products that are USD
usd_products_df = products_online_df.loc[(products_online_df['quote_currency'] == 'USD')].copy().drop(columns=["quote_currency"])
usd_products_df

,id,base_currency
1,WBTC-USD,WBTC
4,GYEN-USD,GYEN
7,OGN-USD,OGN
8,SPELL-USD,SPELL
10,STORJ-USD,STORJ
...,...,...
449,NCT-USD,NCT
454,IDEX-USD,IDEX
455,FORTH-USD,FORTH
460,ANKR-USD,ANKR


In [52]:
# NOTE:  We should probably reindex `usd_products_df` here

In [53]:

def fetch_product_trades(product):
    trades_url = f"{cb_pub}/products/{product}/trades?limit=1000"
    headers = {"Accept": "application/json"}
    trades_columns = ["time","trade_id","price","size","side"]
    response = requests.get(trades_url, headers=headers)

    if response.status_code == 200:
        trades_df = pd.DataFrame(json.loads(response.text), columns=trades_columns)
        trades_df['date'] = pd.to_datetime(trades_df['time'])  # convert to a readable date
        today = datetime.today().strftime('%Y-%m-%d')

        if trades_df is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            filename = f'data/coinbase_{product}_trades_{today}.csv'
            trades_df.to_csv(filename, index=False)
            print(f'Created CSV file: {filename}.')
    else:
        print("Did not receieve OK response from Coinbase API")
        print(response.text)
        
    return


top_stonks = usd_products_df.head(10).sort_values(by=['id'])


for coins in top_stonks['id']:
    fetch_product_trades(coins)
    

Created CSV file: data/coinbase_ALCX-USD_trades_2022-05-14.csv.
Created CSV file: data/coinbase_CRV-USD_trades_2022-05-14.csv.
Created CSV file: data/coinbase_ENJ-USD_trades_2022-05-14.csv.
Created CSV file: data/coinbase_GYEN-USD_trades_2022-05-14.csv.
Created CSV file: data/coinbase_OGN-USD_trades_2022-05-14.csv.
Created CSV file: data/coinbase_PAX-USD_trades_2022-05-14.csv.
Created CSV file: data/coinbase_SPELL-USD_trades_2022-05-14.csv.
Created CSV file: data/coinbase_STORJ-USD_trades_2022-05-14.csv.
Created CSV file: data/coinbase_WBTC-USD_trades_2022-05-14.csv.
Created CSV file: data/coinbase_ZEN-USD_trades_2022-05-14.csv.


In [54]:
# Fetch product ticker into dict()
def fetch_product_ticker(product):
    ticker_url = f"{cb_pub}/products/{product}/ticker"
    headers = {"Accept": "application/json"}
    response = requests.get(ticker_url, headers=headers)

    if response.status_code == 200:
        return dict(json.loads(response.text))
    else:
        print("Did not receieve OK response from Coinbase API")
       
    return

ada_usd_ticker_dict = fetch_product_ticker('ADA-USD')
ada_usd_ticker_dict

{'ask': '0.5265',
 'bid': '0.5263',
 'volume': '91107400.35',
 'trade_id': 70896849,
 'price': '0.5263',
 'size': '453.16',
 'time': '2022-05-14T22:25:36.802688Z'}

In [55]:
# Get a specific product stats
def fetch_product_stats(product):
    stats_url = f"{cb_pub}/products/{product}/stats"
    headers = {"Accept": "application/json"}
    response = requests.get(stats_url, headers=headers)

    if response.status_code == 200:
        return dict(json.loads(response.text))
    else:
        print("Did not receieve OK response from Coinbase API")
    return


ada_usd_stats_dict = fetch_product_stats('ADA-USD')
ada_usd_stats_dict

{'open': '0.5394',
 'high': '0.5565',
 'low': '0.4879',
 'last': '0.528',
 'volume': '91104508.81',
 'volume_30day': '2544775970.7'}

In [56]:
fetch_product_stats('BTC-EUR')

{'open': '28585.22',
 'high': '28724.69',
 'low': '27487.33',
 'last': '28487.28',
 'volume': '1347.51787357',
 'volume_30day': '44529.81255923'}

In [57]:
fetch_product_ticker('BTC-EUR')

{'ask': '28442.2',
 'bid': '28441.69',
 'volume': '1347.65899373',
 'trade_id': 65350210,
 'price': '28446.21',
 'size': '0.0576',
 'time': '2022-05-14T22:25:44.627034Z'}

In [58]:
## Fetch daily candle data and save to CSV file

def fetch_product_candles(product):
    url = f'{cb_pub}/products/{product}/candles?granularity=86400' ## &start={candle_start}&end={candle_end}'
    response = requests.get(url)
    if response.status_code == 200:
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close'] # multiply the BTC volume by closing price to approximate fiat volume
        today = datetime.today().strftime('%Y-%m-%d')

        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            filename = f'data/coinbase_{product}_candles_10_month_{today}.csv'
            data.to_csv(filename, index=False)
            print(f'Created CSV file: {filename}.')
    else:
        print("Did not receieve OK response from Coinbase API")

for coins in top_stonks['id']:
    fetch_product_candles(coins)

Created CSV file: data/coinbase_ALCX-USD_candles_10_month_2022-05-14.csv.
Created CSV file: data/coinbase_CRV-USD_candles_10_month_2022-05-14.csv.
Created CSV file: data/coinbase_ENJ-USD_candles_10_month_2022-05-14.csv.
Created CSV file: data/coinbase_GYEN-USD_candles_10_month_2022-05-14.csv.
Created CSV file: data/coinbase_OGN-USD_candles_10_month_2022-05-14.csv.
Created CSV file: data/coinbase_PAX-USD_candles_10_month_2022-05-14.csv.
Created CSV file: data/coinbase_SPELL-USD_candles_10_month_2022-05-14.csv.
Created CSV file: data/coinbase_STORJ-USD_candles_10_month_2022-05-14.csv.
Created CSV file: data/coinbase_WBTC-USD_candles_10_month_2022-05-14.csv.
Created CSV file: data/coinbase_ZEN-USD_candles_10_month_2022-05-14.csv.
